# CSCI 3155 Recitation 8
October 19, 2018

## Continuations
Continuations are functions we pass as arguments to other functions which capture what to "do next". Instead of writing:
```
def func() {
    val x = doThing1()
    val y = doThing2(x, 3)
    doThing3()
}
```
we can make every function call recursive by changing it to continuation passing style:
```
def func[X, Y](k: X => Y) {
    doThing1(x => DoThing2(x, 3, y => doThing3(k))
}

// Same as:
def func[X, Y](k: X => Y) {
    doThing1(          x =>
    doThing2(x, 3,     y =>
    doThing3(          k))
}
```

### Examples

https://docs.google.com/presentation/d/1OlAu7pjBfKogQg6Z5Ykkid9FfWVH-yrSbPUCHMI-3HM/edit?usp=sharing

### Exercise: Backtracking
As with any good programing tool, we can use continuations to solve problems without putting them everywhere in our code. In this example, you will write a search function that looks for a value in a binary tree, **not** a B**S**T. Take advantage of continuations to remember where to look if something isn't found while keeping the function tail recursive.

In [2]:
sealed trait Tree
case object Empty extends Tree
case class Node(l: Tree, d: Int, r: Tree) extends Tree

def search(t: Tree, i: Int, fail_continuation: () => Boolean): Boolean =
    // BEGIN SOLUTION
    t match {
        case Empty => fail_continuation()
        case Node(l, j, r) if i == j =>
            true
        case Node(l, j, r) =>
            search(l, i, () => {
                search(r, i, () => {
                    fail_continuation()
                })
            })
    }
    //END SOLUTION


defined trait Tree
defined object Empty
defined class Node
defined function search

In [3]:
val t = Node(Empty, 10, Node(Empty, 6, Empty))
assert(search(t, 10, () => false))
assert(!search(t, 0, () => false))

t: Node = Node(Empty,10,Node(Empty,6,Empty))

### Exercise: Eval (again)

This is similar to the example in class: implement eval for the small language given below, ensuring that every function call is a tail call.

In [4]:
sealed trait Expr
case class BoolLiteral(b: Boolean) extends Expr
case class And(left: Expr, right: Expr) extends Expr
case class If(test: Expr, then: Expr, otherwise: Expr) extends Expr

def eval(e: Expr, continuation: Boolean => Boolean): Boolean =
    // BEGIN SOLUTION
    e match {
        case BoolLiteral(b) =>
            continuation(b)
        case And(left, right) =>
            eval(left, left_val =>
                 eval(right, right_val =>
                      continuation(left_val && right_val)
                     )
                )
        case If(test, then, otherwise) =>
            eval(test, test_val =>
                 if (test_val) eval(then, b => continuation(b))
                 else eval(otherwise, continuation)
                )
    }
    // END SOLUTION

defined trait Expr
defined class BoolLiteral
defined class And
defined class If
defined function eval

In [5]:
val e = If(
    And(BoolLiteral(true), BoolLiteral(true)),
    BoolLiteral(false),
    BoolLiteral(true)
)
assert(!eval(e, x => x))

e: If = If(And(BoolLiteral(true),BoolLiteral(true)),BoolLiteral(false),BoolLiteral(true))

## Polymorphism

Polymorphism allows us to write reusable, type safe code. This is also called type parameters, generics, template parameters, or type variables.

As a reminder, we list type parameters in brackets similarly to how we list regular parameters in parentheses:

In [6]:
def id[T](t: T): T = t
// OR
case class C[T](t: T) {
    def get: T = t
}

defined function id
defined class C

Type inference works just the same for these as it does for concrete types, and the scope of type parameters is the function body, like regular parameters:

In [7]:
def f[T](t: T): T = {
    val t2: T = t
    val t3    = t2
    t3
}

defined function f

### Exercise: Reusable data structures
Make a reusable tree data structure which can hold any arbitrary type at it's nodes, as long as all nodes in the tree have the same type. Use the constructors `Empty` and `Node`

In [8]:
sealed trait TreePoly[T]
// BEGIN SOLUTION
case class EmptyPoly[T]() extends TreePoly[T]
case class NodePoly[T](l: TreePoly[T], d: T, r: TreePoly[T]) extends TreePoly[T]
// END SOLUTION

defined trait TreePoly
defined class EmptyPoly
defined class NodePoly

In [9]:
// This should compile:
val t = NodePoly(
    NodePoly(
        EmptyPoly(),
        5,
        EmptyPoly()),
    6,
    EmptyPoly())

t: NodePoly[Int] = NodePoly(NodePoly(EmptyPoly(),5,EmptyPoly()),6,EmptyPoly())

In [9]:
// This should NOT compile:
NodePoly(
    NodePoly(
        EmptyPoly(),
        5,
        EmptyPoly()),
    "6",
    EmptyPoly())

cmd9.sc:2: type mismatch;
 found   : cmd9Wrapper.this.cmd7.wrapper.NodePoly[Int]
 required: cmd9Wrapper.this.cmd7.wrapper.TreePoly[Any]
Note: Int <: Any, but trait TreePoly is invariant in type T.
You may wish to define T as +T instead. (SLS 4.5)
    NodePoly(
            ^

: 

> _Side-note_: for those that are anoyed by the need to make a "new" empty node each time as opposed to before where we could just have one `case object`, we can get around it as follows (see https://stackoverflow.com/questions/7399044/scala-upper-type-bounds-and-parent-classes)
> 
> ```
sealed trait Tree[+T]
case object Empty extends Tree[Nothing]
```

### Exercise: generic functions
Implement map, which should allow someone to take a `Tree[X]` and a function `X => Y` and get out a `Tree[Y]`. Do this with continuations.

In [10]:
// BEGIN SOLUTION
def map[X, Y](t: TreePoly[X], f: X => Y, continuation: TreePoly[Y] => TreePoly[Y]): TreePoly[Y] = t match {
    case EmptyPoly() =>
        continuation(EmptyPoly())
    case NodePoly(l, d, r) =>
        map(l, f, l_mapped =>
            map(r, f, r_mapped =>
                continuation(NodePoly(l_mapped, f(d), r_mapped))))
}
// END SOLUTION

defined function map

In [11]:
// TEST
val t2 = NodePoly(
    NodePoly(
        EmptyPoly(),
        "5",
        EmptyPoly()),
    "6",
    EmptyPoly())

assert(map(t, (i: Int) => i.toString, id[TreePoly[String]]) == t2)
// TEST

t2: NodePoly[String] = NodePoly(NodePoly(EmptyPoly(),5,EmptyPoly()),6,EmptyPoly())